#### Objective : Use Hugging Face Transfomer to fine tune a pre trained model(here we use Distil BERT) and apply on our own dataset
Here use Tersorflow model - tf.kera , not TFTrainer, TFTrainingArguments

In [5]:
import pandas as pd
df=messages = pd.read_csv('SMSSpamCollection', sep='\t',
                           names=["label", "message"])
df.head()


,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [6]:
# independent features
X=list(df['message']) # converted into list
y=list(df['label'])
y[:10]

['ham', 'ham', 'spam', 'ham', 'ham', 'spam', 'ham', 'ham', 'spam', 'spam']

In [7]:
# Convert 'ham' & 'spam' into 0 & 1 , lebel encoding at dependent feature
#pd.get_dummies(y,drop_first=True)
y=list(pd.get_dummies(y,drop_first=True)['spam'])
y[:10]

[0, 0, 1, 0, 0, 1, 0, 0, 1, 1]

In [8]:
# train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

In [ ]:
!pip install transformers
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')


In [10]:
# Train Test text tokenized , encoding, default pad size 512
train_encodings = tokenizer(X_train, truncation=True, padding=True)
test_encodings = tokenizer(X_test, truncation=True, padding=True)

In [11]:
import tensorflow as tf

train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    y_train
))

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    y_test
))

In [4]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy

model.compile(
    optimizer="adam",
    loss=SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)
model.fit(
    tf_train_dataset,
    validation_data=tf_validation_dataset,
)


In [ ]:
from transformers import TFDistilBertForSequenceClassification
model = TFDistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

In [16]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
model.compile(optimizer=optimizer, loss=SparseCategoricalCrossentropy(from_logits=True)) # can also use any keras loss fn
model.fit(train_dataset.shuffle(1000).batch(16), epochs=3, batch_size=16)

Epoch 1/3
279/279 [==============================] - 5986s 21s/step - loss: 0.0740
Epoch 2/3
279/279 [==============================] - 6031s 22s/step - loss: 0.0250
Epoch 3/3
279/279 [==============================] - 6051s 22s/step - loss: 0.0106


In [17]:
model.evaluate(test_dataset)

ValueError: ignored

In [18]:
model.predict(test_dataset)

1115/1115 [==============================] - 541s 483ms/step


TFSequenceClassifierOutput(loss=None, logits=array([[ 3.9972959, -3.4886434],
       [-2.272533 ,  2.4153228],
       [ 4.224286 , -3.6744745],
       ...,
       [ 4.351505 , -3.7816308],
       [-2.2952127,  2.4382513],
       [ 4.11511  , -3.6442838]], dtype=float32), hidden_states=None, attentions=None)

In [19]:
import numpy as np
output = model.predict(test_dataset)[0]
output = np.argmax(output, axis = - 1)

1115/1115 [==============================] - 502s 451ms/step


In [20]:
from sklearn.metrics import confusion_matrix

cm=confusion_matrix(y_test,output)
cm

array([[922,  33],
       [  2, 158]])

In [ ]:
model.save_model('senti_model')